## 6.3 同一性、相等性和别名

对象一旦创建，标识始终不变，可以把标识理解为对象在内存中的地址。is 运算符比较两个对象的标识，id() 函数返回对象的标识。

### 6.3.1 在==和is之间选择

常用is判断变量绑定的值是不是None
x is None
x is not None

is运算符比==快

a==b是语法糖，等同于a.__eq__(b)

一般来说，is运算符只用来判断None，不确定用is还是==时，用==

### 6.3.2 元组的相对不可变性

元组存储的是对象的引用。

In [4]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
print(t1 == t2)

print(id(t1[-1]))
t1[-1].append(99)
print(t1)
print(id(t1[-1]))
print(t1 == t2)

True
4875325184
(1, 2, [30, 40, 99])
4875325184
False


## 6.4 默认做浅拷贝


In [5]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)
print(l2)
print(l1 == l2)
print(l1 is l2)
l3 = l1[:]
print(l3)
print(l3 == l1)
print(l3 is l1)

[3, [55, 44], (7, 8, 9)]
True
False
[3, [55, 44], (7, 8, 9)]
True
False


构造函数或[:]做浅拷贝

In [8]:
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)
l1.append(100)
l1[1].remove(55)
print('l1:', l1)
print('l2:', l2)
# 对于列表来说 += 是就地修改，l2[1] 和 l1[1] 是同一个列表
l2[1] += [33, 22]
# 对于元组来说 += 创造一个新元组，然后重新绑定给变量，等同于l2[2] = l2[2] + (10, 11),现在两个列表中的元组不是同一个了
l2[2] += (10, 11)
print('l1:', l1)
print('l2:', l2)

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]
l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


copy模块的copy()和deepcopy()可以为任意对象做浅拷贝和深拷贝。
一般来说深拷贝并不简单，如果对象有循环引用，那么简单的算法会进入无限循环。
deepcopy()会记住已经复制的对象，因此能优雅地处理循环引用。

In [10]:
a = [10,20]
b = [a,30]
a.append(b)
print(a)
from copy import deepcopy
c = deepcopy(a)
print(c)

[10, 20, [[...], 30]]
[10, 20, [[...], 30]]


## 6.5 函数的参数是引用时

Python传参的模式是共享传参，即函数的各个形式参数获得实参中各个引用的副本。(JS也是)
因此函数可能会修改接收到的任何可变对象。

In [15]:
def f(a, b):
    a += b
    return a

x = 1
y = 2
print(f(x, y))
print(x, y)

a = [1, 2]
b = [3, 4]
print(f(a, b))
print(a, b)

t = (10, 20)
u = (30, 40)
print(f(t, u))
print(t, u)

3
1 2
[1, 2, 3, 4]
[1, 2, 3, 4] [3, 4]
(10, 20, 30, 40)
(10, 20) (30, 40)


### 6.5.1 不要使用可变类型作为参数的默认值


In [27]:
class HauntedBus:
    def __init__(self, passengers=[]):
        self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

bus1 = HauntedBus(['Alice', 'Bill'])
print(bus1.passengers)
bus1.pick('Charlie')
bus1.drop('Alice')
print(bus1.passengers)


bus2 = HauntedBus()
bus2.pick('Carrie')
print(bus2.passengers)


bus3 = HauntedBus()
print(bus3.passengers)
bus3.pick('Dave')

print(bus2.passengers)
print(bus2.passengers is bus3.passengers)

print(bus1.passengers)

print(dir(HauntedBus.__init__))
print(HauntedBus.__init__.__defaults__)
print(HauntedBus.__init__.__defaults__[0] is bus2.passengers)

['Alice', 'Bill']
['Bill', 'Charlie']
['Carrie']
['Carrie']
['Carrie', 'Dave']
True
['Bill', 'Charlie']
['__annotations__', '__builtins__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__getstate__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
(['Carrie', 'Dave'],)
True


没有指定初始乘客的HauntedBus实例会共享同一个乘客列表。
self.passengers变成了passengers的别名，而不是副本。
默认值在定义函数时求解，因此默认值变成了函数对象的属性。所以如果默认值是可变对象，而且修改了它的值，那么后续的函数调用都会受到影响。

### 6.5.2 防御可变参数

如果函数收到的参数是可变的，应该谨慎考虑是否要修改传入的参数。

In [28]:
class TwilightBus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers
            # 正确做法是做复制
            # self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
print(basketball_team)

['Sue', 'Maya', 'Diana']


## 6.6 del和垃圾回收

对象绝不会自行销毁，然而，无法得到对象时，可能会被垃圾回收。

del x 等同于 del(x)
del语句删除引用，而不是对象。del可能导致对象被当作垃圾回收，但是仅当删除的变量保存的是对象的最后一个引用。
重新绑定也可能导致对象的引用数量归零，导致对象被销毁。

In [29]:
a = [1,2]
b = a
# 此时 [1,2]还有b的引用
del a
print(b)
# 此时垃圾回收器会回收[1,2]的内存
b = [3]

[1, 2]


在CPython中，垃圾回收使用的主要算法是引用计数。当引用计数归零时，对象立即就被销毁：CPython会在对象上调用__del__方法，然后释放分配给对象的内存。
CPython2.0增加了分代垃圾回收算法，用于检测引用循环中涉及的对象组。

In [31]:
import weakref
s1 = {1, 2, 3}
s2 = s1
def bye():
    print('Gone with the wind...')

ender = weakref.finalize(s1, bye)
print(ender.alive)
del s1
print(ender.alive)
s2 = 'spam'
print(ender.alive)


Gone with the wind...
True
True
Gone with the wind...
False


finalize是对{1, 2, 3}的弱引用，弱引用不增加对象的引用计数。

## 6.8 小结

对+=或*=所做的增量赋值来说，如果左边的变量绑定的是不可变对象，会生成新对象，如果是可变对象，会就地修改。